In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix

mort = pd.read_csv('../data/data.csv', header=0) # for train
test = pd.read_csv('../data/datatest.csv', header=0) # for test
mort

,Bo_Age,Ln_Orig,Orig_LTV_Ratio_Pct,Credit_score,First_home,Tot_mthly_debt_exp,Tot_mthly_incm,orig_apprd_val_amt,pur_prc_amt,DTI Ratio,Status,OUTCOME,State,Median_state_inc,UPB>Appraisal
0,37,75650.0,85,669,N,1707,6000,169000,160000,0.284500,Active,non-default,FL,"40,171",0
1,46,390775.0,102,684,N,0,5025,310000,309000,0.000000,Pay-off,non-default,NY,"44,228",1
2,30,112500.0,90,662,Y,1812,4800,177000,176450,0.377500,Pay-off,non-default,CA,"49,894",0
3,24,85250.0,97,647,N,3395,6934,113000,110000,0.489616,Pay-off,non-default,GA,"43,217",0
4,35,114000.0,100,791,N,3801,5504,103000,103000,0.690589,Pay-off,non-default,NH,"57,352",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10602,37,280200.0,95,652,Y,727,1827,80000,80000,0.397920,Pay-off,non-default,AZ,"42,590",1
10603,30,57000.0,94,589,N,1703,3941,109000,108000,0.432124,Pay-off,non-default,IL,"45,787",0
10604,33,153200.0,95,675,Y,1726,3707,154000,154000,0.465606,Pay-off,non-default,GA,"43,217",0
10605,38,158850.0,100,786,Y,1364,2500,92000,92000,0.545600,Active,non-default,FL,"40,171",1


In [2]:
###### mort 전처리 ######

val = len(mort)

### 추가 DATA set 만들기
mort['Credit'] = np.nan # CS<620 -> 1
mort['LTV'] = np.nan # LTV>75 -> 1
mort['DTI'] = np.nan # DTI>43 -> 1
mort['Total'] = np.nan # cs & ltv & dti -> 1

for i in range(0,val):
    if mort['Credit_score'][i] > 620:
        mort['Credit'][i] = 0
    else:
        mort['Credit'][i] = 1

for j in range(0,val):
    if mort['Orig_LTV_Ratio_Pct'][j] > 75:
        mort['LTV'][j] = 1
    else:
        mort['LTV'][j] = 0

for k in range(0,val):
    if mort['DTI Ratio'][k] > 0.43:
        mort['DTI'][k] = 1
    else:
        mort['DTI'][k] = 0

for l in range(0,val):
    if mort['Credit'][l] == 1 and mort['LTV'][l] == 1 and mort['DTI'][l] == 1:
        mort['Total'] = 1
    else:
        mort['Total'] = 0

### 전처리 과정
## First_home 문자형 데이터 변환
mort['First_home'] = np.where(mort['First_home']=='Y',1,0)

## OUTCOME 데이터 변환
mort['OUTCOME'] = np.where(mort['OUTCOME']=='non-default',0,1) # non-default = 0, default = 1

## string 타입을 정수로 변환
for k in range(0,val):
    mort.Median_state_inc[k] = np.float64(mort.Median_state_inc[k].replace(",", ""))
    
# Separating 'target' feature
outcome = mort.OUTCOME
mort.drop(['OUTCOME'], axis='columns', inplace=True)

## Delete feature
mort = mort.drop('Status', axis=1)
mort = mort.drop('State', axis=1)
mort = mort.drop('orig_apprd_val_amt', axis=1)
mort = mort.drop('UPB>Appraisal', axis=1)
##
# mort = mort.drop('Orig_LTV_Ratio_Pct', axis=1)
# mort = mort.drop('Credit_score', axis=1)
# mort = mort.drop('DTI Ratio', axis=1)

mort

c:\anaconda3\envs\dm2019\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
c:\anaconda3\envs\dm2019\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
c:\anaconda3\envs\dm2019\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\anaconda3\envs\dm2019\lib\site-packages\i

,Bo_Age,Ln_Orig,Orig_LTV_Ratio_Pct,Credit_score,First_home,Tot_mthly_debt_exp,Tot_mthly_incm,pur_prc_amt,DTI Ratio,Median_state_inc,Credit,LTV,DTI,Total
0,37,75650.0,85,669,0,1707,6000,160000,0.284500,40171,0.0,1.0,0.0,0
1,46,390775.0,102,684,0,0,5025,309000,0.000000,44228,0.0,1.0,0.0,0
2,30,112500.0,90,662,1,1812,4800,176450,0.377500,49894,0.0,1.0,0.0,0
3,24,85250.0,97,647,0,3395,6934,110000,0.489616,43217,0.0,1.0,1.0,0
4,35,114000.0,100,791,0,3801,5504,103000,0.690589,57352,0.0,1.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10602,37,280200.0,95,652,1,727,1827,80000,0.397920,42590,0.0,1.0,0.0,0
10603,30,57000.0,94,589,0,1703,3941,108000,0.432124,45787,1.0,1.0,1.0,0
10604,33,153200.0,95,675,1,1726,3707,154000,0.465606,43217,0.0,1.0,1.0,0
10605,38,158850.0,100,786,1,1364,2500,92000,0.545600,40171,0.0,1.0,1.0,0


In [3]:
###### test 전처리 ######

val = len(test)

### 추가 DATA set 만들기
test['Credit'] = np.nan # CS<620 -> 1
test['LTV'] = np.nan # LTV>75 -> 1
test['DTI'] = np.nan # DTI>43 -> 1
test['Total'] = np.nan # cs & ltv & dti -> 1

for i in range(0,val):
    if test['Credit_score'][i] > 620:
        test['Credit'][i] = 0
    else:
        test['Credit'][i] = 1

for j in range(0,val):
    if test['Orig_LTV_Ratio_Pct'][j] > 75:
        test['LTV'][j] = 1
    else:
        test['LTV'][j] = 0

for k in range(0,val):
    if test['DTI Ratio'][k] > 0.43:
        test['DTI'][k] = 1
    else:
        test['DTI'][k] = 0

for l in range(0,val):
    if test['Credit'][l] == 1 and test['LTV'][l] == 1 and test['DTI'][l] == 1:
        test['Total'] = 1
    else:
        test['Total'] = 0
        
## First_home 문자형 데이터 변환
test['First_home'] = np.where(test['First_home']=='Y',1,0)

## string 타입을 정수로 변환
for k in range(0,val):
    test.Median_state_inc[k] = np.float64(test.Median_state_inc[k].replace(",", ""))

## Delete feature
test = test.drop('Status', axis=1)
test = test.drop('State', axis=1)
test = test.drop('orig_apprd_val_amt', axis=1)
test = test.drop('UPB>Appraisal', axis=1)

test

c:\anaconda3\envs\dm2019\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
c:\anaconda3\envs\dm2019\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
c:\anaconda3\envs\dm2019\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\anaconda3\envs\dm2019\lib\site-packages\i

,Bo_Age,Ln_Orig,Orig_LTV_Ratio_Pct,Credit_score,First_home,Tot_mthly_debt_exp,Tot_mthly_incm,pur_prc_amt,DTI Ratio,Median_state_inc,Credit,LTV,DTI,Total
0,37,83247.0,95,760,1,1791,3640,164900,0.492033,51022,0.0,1.0,1.0,0
1,31,67000.0,100,708,0,1233,4485,67000,0.274916,47220,0.0,1.0,0.0,0
2,44,226900.0,100,656,1,1497,3146,109900,0.475842,53275,0.0,1.0,1.0,0
3,27,77500.0,100,747,1,2563,5832,167900,0.439472,41275,0.0,1.0,1.0,0
4,28,100000.0,100,544,0,975,4669,154250,0.208824,40171,1.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4541,49,211638.0,102,625,0,1701,3033,139725,0.560831,43217,0.0,1.0,1.0,0
4542,37,99200.0,80,661,0,1446,3481,144000,0.415398,39326,0.0,1.0,0.0,0
4543,36,136400.0,99,650,1,2990,5876,170000,0.508850,40171,0.0,1.0,1.0,0
4544,46,29900.0,100,562,0,2854,7830,185000,0.364496,43003,1.0,1.0,0.0,0


In [4]:
X_train = mort
y_train = outcome
X_test = test

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from imblearn.over_sampling import SMOTE
sm = SMOTE(ratio='auto', kind='regular')
X_train, y_train = sm.fit_sample(X_train, y_train)

pd.Series(y_train).value_counts()

1    10325
0    10325
dtype: int64

In [5]:
from sklearn.feature_selection import SelectKBest, f_classif
selector = SelectKBest(f_classif, k = 7).fit(X_train, y_train)
X_train = selector.transform(X_train)
X_test = selector.transform(X_test)

c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [13] are constant.
  UserWarning)
c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [6]:
forest = RandomForestClassifier(criterion='gini', n_estimators=500, max_leaf_nodes=16, random_state=1) # 추가 DATA
forest.fit(X_train, y_train)

pred = forest.predict(X_test)
pred

array([0, 1, 0, ..., 0, 1, 1])

In [7]:
type(pred)

numpy.ndarray

In [8]:
df = pd.DataFrame(pred)

In [9]:
df.to_excel("output.xlsx")